# Getting Started

Scryfall API:
"We kindly ask that you insert 50 – 100 milliseconds of delay between the requests you send to the server. (i.e., 10 requests per second on average)."

In [ ]:
# import all the things
import numpy as np
import pandas as pd
import json, requests, pickle
from bs4 import BeautifulSoup

In [ ]:
# Get setlist into dataframe
link = 'https://api.scryfall.com/sets'
response = requests.get(link)
sets = response.json()['data']
sets_df = pd.DataFrame(sets)

## Clean up set data 

In [ ]:
# Ignore sets with no release date (likely promotional / outliers), set release date to index
sets_df.dropna(subset=['released_at'], inplace=True)
sets_df.set_index('released_at', inplace=True)

# Remove repetitive information and online data
sets_df.drop(['digital','mtgo_code','parent_set_code','object'], axis=1, inplace=True)

In [ ]:
# Make even cleaner for ease of exploration
clean_df = sets_df[['name','code','block_code','card_count','set_type']]
clean_df['set_type'].unique()

In [ ]:
# after extensive manual checking of set legality / onlineness
sets_to_drop = [
    'me1',
    'me2',
    'me3',
    'me4',
    'vma',
    'tpr',
    'e02',
    'gnt',
    'td0',
    'mgb',
    'ana',
    'w17',
    'w16',
    'itp'
]
types_to_drop = [
    'memorabilia',
    'funny',
    'treasure_chest'
]

In [ ]:
final_sets = sets_df[sets_df['code'].apply(lambda x: x not in sets_to_drop)]
final_sets = final_sets[final_sets['set_type'].apply(lambda x: x not in types_to_drop)]
final_sets.head()

## Get Card Data

In [ ]:
# get 1 page
link = 'https://api.scryfall.com/cards?page=1'
response = requests.get(link)
cards = response.json()['data']
cards_df = pd.DataFrame(cards)

In [ ]:
cards_df.info()

In [ ]:
# Filters: not legal in vintage (tokens, joke cards, conspiracies, etc.), only english cards
clean_cards = cards_df[(cards_df['legalities'].apply(lambda x: x['vintage']!='not_legal')) & (cards_df['lang']=='en')]
clean_cards.set_index('id', inplace=True)
clean_cards.info()

In [ ]:
# Features to keep
MVP_features = [
    'name',
    'set_name',
    'type_line',    
    'mana_cost',
    'rarity',
    'oracle_text',
    'power',
    'toughness',
    'loyalty',
    'cmc',
    'set',
    'color_identity',
    'colors',    
    'reprint',
    'layout',
    'legalities',
]

misc_features = [
    'all_parts',
    'artist',
    'border_color',
    'card_faces',
    'edhrec_rank',
    'flavor_text',
    'foil',
    'nonfoil',
    'full_art',
    'watermark'    
    'timeshifted',
    'colorshifted',
    'futureshifted',
    'illustration_id',
    'multiverse_ids',
    'oracle_id',
    'prints_search_uri',
    'rulings_uri',
    'set_search_uri',
]

In [ ]:
clean_cards[MVP_features].head()

Next Step: Write loop into scraper, get all the cards! Get the hell off jupyter ntoebooK!

### Formats
Starting w/ modern:
* Standard
* Modern
* Extended
* Legacy
* Vintage
* Block Constructed (deprecated)
* Extended (deprecated)
* Commander
* Casual

Questions:
    Should I only care about non foils?
    Should I do reprint / set search on my own, or use the API? 

## Post-Scryfall Scrape: How do the cards look?

In [ ]:
all_cards_df = pd.read_csv('all_vintage_cards.csv')
all_cards_df.info()

## MTGPrice Scraping

In [ ]:
# trying slimit parser
from slimit import ast
from slimit.parser import Parser
from slimit.visitors import nodevisitor

In [ ]:
# Turn card data into soup
link = 'https://www.mtgprice.com/sets/Visions/Vampiric_Tutor'
soup = BeautifulSoup(requests.get(link).content, 'html.parser')

# GET RESULTS
text_to_find = 'var results = ['
history=[]
for script in soup.findAll('script', type='text/javascript'):
    if text_to_find in script.text:
        parser = Parser()
        tree = parser.parse(script.text)
        for node in nodevisitor.visit(tree):
            if isinstance(node, ast.Assign) and getattr(node.left, 'value', '') == "\"data\"":
                for prices in node.right.items:
                    history.append([prices.items[0].value,prices.items[1].value])
                break
print(np.array(history).shape)

### Let's try a whole set

In [ ]:
def card_price_history(setname, cardname):
    # Turn card data into soup
    link = 'https://www.mtgprice.com/sets/' + '_'.join(setname.split()) + '/' + '_'.join(cardname.split())
    soup = BeautifulSoup(requests.get(link).content, 'html.parser')

    # GET RESULTS
    text_to_find = 'var results = ['
    history=[]
    for script in soup.findAll('script', type='text/javascript'):
        if text_to_find in script.text:
            parser = Parser()
            tree = parser.parse(script.text)
            for node in nodevisitor.visit(tree):
                if isinstance(node, ast.Assign) and getattr(node.left, 'value', '') == "\"data\"":
                    for prices in node.right.items:
                        history.append([prices.items[0].value,prices.items[1].value])
                    break
    return np.array(history)

In [ ]:
def sets_price_history(sets, all_cards_df):
    set_dict = {}
    for setname in sets:
        print(setname)
        cards = all_cards_df[all_cards_df['set_name'] == setname]['name'].values
        card_dict = {}
        for cardname in cards:
            if '/' in cardname:
                cardname = cardname.split('/')[0]
            print(cardname)
            try:
                history = card_price_history(setname, cardname)
                card_dict[cardname] = history
            except:
                print('{} not a set on MTGPrice'.format(setname))
                break
        set_dict[setname] = card_dict

In [ ]:
with open('price_scrape_0.p', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    MVP_scrape = u.load()

In [ ]:
sets_price_history(sets, all_cards_df)